# Markdown → Google Doc (Colab)

This notebook authenticates with Google APIs and converts the provided Markdown meeting notes into a well‑formatted Google Doc with headings, bullets, checkboxes, assignee highlights, and styled footer.

In [27]:
# Install dependencies
!pip -q install google-api-python-client==2.147.0

In [28]:
from google.colab import auth  # type: ignore
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re
from typing import Optional, Tuple

print("Google Authentication complete.")

Google Authentication complete.


In [29]:
# Load Markdown file. Upload your file to /content/meeting.md in Google Colab
import os
MD_PATH = "/content/meeting.md"
if not os.path.exists(MD_PATH):
    # Paste markdown as-is as a fallback if file upload doesn't work.
    md_text = "Please upload meeting.md to /content first."
else:
    with open(MD_PATH, 'r', encoding='utf-8') as f:
        md_text = f.read()
print("Loaded markdown with", len(md_text.splitlines()), "lines")

Loaded markdown with 112 lines


In [30]:
# === Regular Expressions ===
# Matches headings like "# Title" or "### Subsection"
HEADER_RE = re.compile(r'^(#{1,6})\s+(.*)$')
# Matches markdown checkboxes like "- [ ] Task item"
CHECKBOX_RE = re.compile(r'^\s*[-*]\s*\[\s*\]\s+(.*)$')
# Matches bullet points like "- item" or "* item"
BULLET_RE = re.compile(r'^(\s*)[-*]\s+(.*)$')
# Matches horizontal rules "---", "***", or "___"
HR_RE = re.compile(r'^\s{0,3}(-{3,}|\*{3,}|_{3,})\s*$')
# Matches footer lines (case-insensitive) like "Meeting recorded by:" or "Duration:"
FOOTER_RE = re.compile(r'^\s*(meeting recorded by:|duration:)\s*', re.I)

def indent_level(line: str) -> int:
    """
    Calculate the indentation level based on leading spaces.
    Markdown commonly uses 2 spaces per indent level.
    """
    leading = len(line) - len(line.lstrip(' '))
    return leading // 2

def parse_markdown(md: str):
    """
    Parse markdown text line-by-line and yield tokens representing structure.

    Each yielded tuple looks like:
        (token_type, data_dict)

    Example:
        ('h2', {'text': 'Agenda'})
    """
    for raw in md.splitlines():
        line = raw.rstrip('\n')

        # Blank lines → signal new paragraph separation
        if not line.strip():
            yield ('blank', {})
            continue

        # === Check for specific markdown structures ===

        # Headings like "# Title" or "## Section"
        m = HEADER_RE.match(line)
        if m:
            level = len(m.group(1))
            text = m.group(2).strip()
            yield (f'h{level}', {'text': text})
            continue

        # Horizontal rule ("---", "***", "___")
        if HR_RE.match(line):
            yield ('hr', {})
            continue

        # Footer lines (Meeting recorded by / Duration)
        if FOOTER_RE.match(line):
            yield ('footer', {'text': line.strip()})
            continue

        # Checkbox task list "- [ ] item"
        m = CHECKBOX_RE.match(line)
        if m:
            yield ('checkbox', {'text': m.group(1).strip(), 'indent': indent_level(raw)})
            continue

        # Regular bullet point "- item"
        m = BULLET_RE.match(line)
        if m:
            spaces, text = m.groups()
            yield ('bullet', {'text': text, 'indent': indent_level(spaces)})
            continue

        # Default: plain text line
        yield ('text', {'text': line})


In [31]:
def new_doc(title: str):
    # Create a new Google Doc with the given title
    docs = build('docs', 'v1')
    return docs.documents().create(body={'title': title}).execute()

def push_updates(document_id: str, requests: list):
    # Push batch update requests to the target document
    docs = build('docs', 'v1')
    return docs.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

def style_range(start, end, heading=None, bold=False, italic=False, color_rgb=None, font_size_pt=None):
    # Apply paragraph and text-level styles
    reqs = []
    if heading:
        reqs.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'paragraphStyle': {'namedStyleType': heading},
                'fields': 'namedStyleType'
            }
        })
    if bold or italic or color_rgb or font_size_pt:
        text_style = {}
        fields = []
        if bold:
            text_style['bold'] = True; fields.append('bold')
        if italic:
            text_style['italic'] = True; fields.append('italic')
        if color_rgb:
            text_style['foregroundColor'] = {'color': {'rgbColor': {
                'red': color_rgb[0], 'green': color_rgb[1], 'blue': color_rgb[2]
            }}}
            fields.append('foregroundColor')
        if font_size_pt:
            text_style['fontSize'] = {'magnitude': font_size_pt, 'unit': 'PT'}
            fields.append('fontSize')
        reqs.append({
            'updateTextStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'textStyle': text_style,
                'fields': ','.join(fields)
            }
        })
    return reqs

def bullet_paragraph_range(start, end, checkbox=False, indent_level_pts=0):
    # Create bullet or checkbox list; apply indent if needed
    reqs = []
    preset = 'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
    reqs.append({
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': preset
        }
    })
    if indent_level_pts:
        reqs.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'paragraphStyle': {'indentStart': {'magnitude': indent_level_pts, 'unit': 'PT'}},
                'fields': 'indentStart'
            }
        })
    return reqs

def insert_text_requests(at, text):
    # Insert plain text at the given index
    return [{'insertText': {'location': {'index': at}, 'text': text}}]

def highlight_assignees(text_start, text, color=(0.13, 0.44, 0.80)):
    # Highlight @mentions in bold blue
    reqs = []
    for m in re.finditer(r'@([A-Za-z0-9_\.-]+)', text):
        s = text_start + m.start(); e = text_start + m.end()
        reqs.extend(style_range(s, e, bold=True, color_rgb=color))
    return reqs

def spacing_range(start, end, space_above=6, space_below=6):
    # Control paragraph spacing (vertical gaps)
    return [{
        'updateParagraphStyle': {
            'range': {'startIndex': start, 'endIndex': end},
            'paragraphStyle': {
                'spaceAbove': {'magnitude': space_above, 'unit': 'PT'},
                'spaceBelow': {'magnitude': space_below, 'unit': 'PT'},
            },
            'fields': 'spaceAbove,spaceBelow'
        }
    }]


In [32]:
title_for_doc = "Product Team Sync"
try:
    doc = new_doc(title_for_doc)
    document_id = doc['documentId']
    print(f"Created Doc: https://docs.google.com/document/d/{document_id}/edit")
    idx = 1
    requests = []

    def add_paragraph(text: str, heading: Optional[str] = None, bold=False, italic=False,
                    color=None, font_size=None) -> Tuple[int, int]:
        global idx, requests
        txt = text + "\n"
        start = idx
        requests.extend(insert_text_requests(idx, txt))
        idx += len(txt)
        if heading or bold or italic or color or font_size:
            requests.extend(
                style_range(start, start + len(txt), heading=heading,
                            bold=bold, italic=italic, color_rgb=color, font_size_pt=font_size)
            )
        requests.extend(highlight_assignees(start, txt))
        return start, start + len(txt)


    last_bullet_block = []

    def flush_bullets(pad_below_pt: int = 0):
        global last_bullet_block, requests
        if not last_bullet_block:
            return
        # Apply bullets for each line
        for (s, e, is_cb, indent_pts) in last_bullet_block:
            requests.extend(bullet_paragraph_range(s, e, checkbox=is_cb, indent_level_pts=indent_pts))
        # Add spacing *below* the final bullet paragraph if requested
        if pad_below_pt > 0:
            last_s, last_e, _, _ = last_bullet_block[-1]
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': last_s, 'endIndex': last_e},
                    'paragraphStyle': {'spaceBelow': {'magnitude': pad_below_pt, 'unit': 'PT'}},
                    'fields': 'spaceBelow'
                }
            })
        last_bullet_block = []


    for t, data in parse_markdown(md_text):
        if t == 'blank':
            # No extra empty paragraphs → avoids big gaps
            flush_bullets()
            continue

        elif t in ('h1','h2','h3','h4','h5','h6'):
            flush_bullets(pad_below_pt=8)
            level = int(t[1])

            if level == 1:
                s, e = add_paragraph(data['text'], heading='HEADING_1')
                # Big top title spacing
                requests.extend(spacing_range(s, e, space_above=12, space_below=10))

            elif level == 2:
                s, e = add_paragraph(data['text'], heading='HEADING_2')
                # Clear gap BEFORE and a little AFTER H2 (e.g., between “Action Items” → “Next Steps”)
                requests.extend(spacing_range(s, e, space_above=12, space_below=6))

            elif level == 3:
                s, e = add_paragraph(data['text'], heading='HEADING_3')
                requests.extend(spacing_range(s, e, space_above=8, space_below=4))

            else:
                s, e = add_paragraph(data['text'])
                requests.extend(spacing_range(s, e, space_above=6, space_below=4))


        elif t in ('bullet', 'checkbox'):
            text = data['text']
            start, end = add_paragraph(text)
            indent_pts = data.get('indent', 0) * 18
            last_bullet_block.append((start, end, t == 'checkbox', indent_pts))

        elif t == 'hr':
            flush_bullets()
            add_paragraph("———")

        elif t == 'footer':
            # Add footer here with styling
            flush_bullets()
            add_paragraph(data['text'], italic=True, color=(0.45, 0.45, 0.45), font_size=10)

        else:  # Plain text
            flush_bullets()
            add_paragraph(data['text'])

    flush_bullets()


    push_updates(document_id, requests)
    print("Document formatted.")
    print(f"Open your doc: https://docs.google.com/document/d/{document_id}/edit")
except HttpError as e:
    print("Google API error:", e)
except Exception as ex:
    print("Unexpected error:", ex)


Created Doc: https://docs.google.com/document/d/1BXfvVStZwIZtMEdlOKLAlYq8j_RncBLDpfFUTZ0Jdb8/edit
✅ Document formatted.
Open your doc: https://docs.google.com/document/d/1BXfvVStZwIZtMEdlOKLAlYq8j_RncBLDpfFUTZ0Jdb8/edit
